<a href="https://colab.research.google.com/github/Potdooshami/2H_TaSe2_Tc_STM/blob/main/Lawler_Fujita(78K_160nm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataprocessing environment setting

In [ ]:
!pip install gwyfile
from google.colab import drive
drive.mount('/content/drive')

load `.gwy` files' list

In [ ]:
dir = '/content/drive/MyDrive/2HDWN/phaseAnalysis'
import os
files = os.listdir(dir)
files_no_ext = [f.replace('.gwy', '') for f in files]

load `.gwy` files' header

In [ ]:
import pandas as pd
fnTbl = '/content/drive/MyDrive/2HDWN/dataTbl.xlsx'
df = pd.read_excel(fnTbl)
df['fileName_no_ext'] = df['fileName_corrected_ntn'].str.replace('.sxm', '', regex=False)
df = df.set_index('fileName_no_ext')
dfCrrent =df.loc[files_no_ext]
dfCrrent = dfCrrent.iloc[[0,1,2,3,4,6,7,8]] #remove redundancy
dfCrrent

# Choose a file

In [ ]:
dfCrrent[['scan_range_1_sxm','bias_sxm', 'scan_pixels_1_sxm']]

In [ ]:
fn = '2HTaSe2_bad_78K097'# @param ['2HTaSe2_ah035','2HTaSe2_ao_115K037','2HTaSe2_aj036','2HTaSe2_ap_118K002','2HTaSe2_bae_110K012','2HTaSe2_bae_110K013','2HTaSe2_bad_78K097','2HTaSe2_bad_78K100']
fnlong = dir +'/'+ fn +'.gwy'

In [ ]:
sz_nano = dfCrrent.loc[fn][['scan_range_1_sxm','scan_range_2_sxm']].values *1000000000
pxl = dfCrrent.loc[fn][['scan_pixels_1_sxm','scan_pixels_2_sxm']].values

## Declare 1st array

In [ ]:
import gwyfile
import numpy as np

# gwy open
container = gwyfile.load(fnlong)
arr = container['/0/data']['data']
arr = arr.reshape(pxl)

In [ ]:
import matplotlib.pyplot as plt
p = (10**(-12))
plt.figure(figsize=(20, 20)) # Adjust the size as needed
img = plt.imshow(arr,cmap='afmhot',vmin=-13*p,vmax=18*p)

Okay, DWN is well defined but large scale trivial landscape and

In [ ]:
plt.imshow(arr,cmap='afmhot')
from scipy.ndimage import gaussian_filter

we can check here the cluster effect

# Remove background

### High-pass filtering

In [ ]:
def ceilcut(image,threshold):
  imagec = image.copy()
  imagec[image>threshold] = threshold
  return imagec
from scipy.ndimage import gaussian_filter

In [ ]:
param_ceilCut = 10 # @param
param_sigma = 5 # @param
from scipy.ndimage import gaussian_filter
arr_c = ceilcut(arr,p*param_ceilCut)
arr_c_g = gaussian_filter(arr_c,sigma=param_sigma)
arr_c_g_m = arr - arr_c_g

####Visualization

In [ ]:
def plot_result(image, background):
    fig, ax = plt.subplots(nrows=1, ncols=3)

    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Original image')
    ax[0].axis('off')

    ax[1].imshow(background, cmap='gray')
    ax[1].set_title('Background')
    ax[1].axis('off')

    ax[2].imshow(image - background, cmap='gray')
    ax[2].set_title('Result')
    ax[2].axis('off')

    fig.tight_layout()
    return fig,ax


##### Original

In [ ]:
plt.figure(figsize=(20, 20)) # Adjust the size as needed
plt.imshow(arr,cmap='afmhot',vmin=-13*p,vmax=18*p)

#####Compares

In [ ]:
plot_result(arr, arr_c_g)

##### Filtered

In [ ]:
plt.figure(figsize=(20, 20)) # Adjust the size as needed

plt.imshow(arr_c_g_m,cmap='afmhot',vmin=-13*p,vmax=18*p)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])

### Cluster Removing

In [ ]:
import numpy as np

def replace_with_average(img_original, bw):
  """
  Replaces the value of specific part of image as average.

  Args:
    img_original: (n,m) float array, the original image.
    bw: (n,m) boolean array, the mask where True indicates the part to be replaced.

  Returns:
    img_cleaned: (n,m) float array, the image with the specified part replaced by the average.
  """
  # Calculate the average of the values in img_original where bw is False
  average_false_part = np.mean(img_original[~bw])

  # Create a copy of the original image
  img_cleaned = img_original.copy()

  # Replace the values in img_cleaned where bw is True with the calculated average
  img_cleaned[bw] = average_false_part

  return img_cleaned

In [ ]:
cutted =  arr_c_g_m>p*5


In [ ]:
from skimage.morphology import binary_opening, disk

# Define the size of the structuring element (adjust as needed)
selem_size = 6 # Example size, you might need to tune this
selem = disk(selem_size)

# Apply the binary opening operation
opened = binary_opening(cutted, footprint=selem)

In [ ]:
opened_rm = opened.copy()
opened_rm[0:100,:] = False

In [ ]:
arr_c_g_mm = replace_with_average(arr_c_g_m , opened_rm)

In [ ]:
# plt.figure(figsize=(20, 20)) # Adjust the size as needed
# plt.imshow(cutted,cmap='gray')
# ax = plt.gca()
# ax.set_xticks([])
# ax.set_yticks([])

In [ ]:
# plt.figure(figsize=(20, 20)) # Adjust the size as needed
# plt.imshow(opened,cmap='gray')
# ax = plt.gca()
# ax.set_xticks([])
# ax.set_yticks([])

In [ ]:
# plt.figure(figsize=(20, 20)) # Adjust the size as needed
# plt.imshow(opened_rm,cmap='gray')
# ax = plt.gca()
# ax.set_xticks([])
# ax.set_yticks([])


In [ ]:
plt.figure(figsize=(20, 20)) # Adjust the size as needed

plt.imshow(arr_c_g_mm,cmap='afmhot')
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])

In [ ]:
# Define a threshold for identifying salt-like noise (adjust as needed)
# This threshold might need tuning based on the characteristics of your noise
noise_threshold = 150 * p # Example threshold, adjust as necessary based on your data

# Create a boolean mask where True indicates potential noise (values above the threshold)
noise_mask = arr_c_g_mm > noise_threshold

# Use the replace_with_average function to replace the masked noise with the average of the rest of the image
arr_cleaned_noise = replace_with_average(arr_c_g_mm, noise_mask)

# Display the cleaned image
plt.figure(figsize=(20, 20))
plt.imshow(arr_cleaned_noise, cmap='afmhot') # You might want a different colormap
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])
plt.title('Image after removing salt-like noise')
plt.show()

### Remove tip tick

In [ ]:
plt.figure(figsize=(20, 20)) # Adjust the size as needed

plt.imshow(arr_c_g_mm>100*p,cmap='afmhot')
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])

In [ ]:
def imshowL(arr):
  plt.figure(figsize=(20, 20)) # Adjust the size as needed
  plt.imshow(arr,cmap='afmhot')
  ax = plt.gca()
  ax.set_xticks([])
  ax.set_yticks([])

In [ ]:
# Perform a circular shift of arr_c_g_mm along axis=0 by 1 pixel
arr_shifted = np.roll(arr_c_g_mm, shift=300, axis=0)

# Display the shifted array (optional)
imshowL(arr_shifted)

In [ ]:
arr_shifted = np.roll(arr_c_g_mm, shift=1, axis=0)
arr2 = arr_c_g_mm - arr_shifted
arr3 = arr2>30*p
arr_c_g_mmm = replace_with_average(arr_c_g_mm, arr3)
plt.figure(figsize=(20, 20)) # Adjust the size as needed
plt.imshow(arr_c_g_mmm,cmap='afmhot',vmin=-13*p,vmax=18*p)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])

In [ ]:
arr_c_g_mmmm = arr_c_g_mmm - np.mean(arr_c_g_mmm)

In [ ]:
# from skimage.measure import label, regionprops

# # Label connected components in the opened image
# labeled_image = label(opened)

# # Get properties of each labeled region
# regions = regionprops(labeled_image)

# print(f"Number of segmented objects: {len(regions)}")

phasemap

# FFT peak find

In [ ]:
from scipy.fft import fft2, fftshift
from skimage.feature import peak_local_max
def fft2show(arr_cln,vmin,vmax):
  fft_result = fft2(arr_cln)
  fft_result_shifted = fftshift(fft_result)
  magnitude_spectrum = np.abs(fft_result_shifted)
  plt.figure(figsize=(10, 10))
  plt.imshow(magnitude_spectrum, cmap='gray', vmin=vmin, vmax=vmax)
  plt.title('Magnitude Spectrum of 2D FFT')
  return  magnitude_spectrum
def fft2pkfnd(fft2abs,threshold,choose):
  coordinates = peak_local_max(fft2abs, min_distance=100, threshold_abs = threshold)
  plt.scatter(coordinates[:, 1], coordinates[:, 0], s=50, facecolors='none', edgecolors='r')
  for ipeak in range(coordinates.shape[0]):
    plt.text(coordinates[ipeak,1],coordinates[ipeak,0],str(ipeak), color='g')
  coordinates_choose = coordinates[choose,:]
  print(coordinates_choose)
  print(coordinates_choose.shape[0])
  for ipeak in range(coordinates_choose.shape[0]):
    plt.text(coordinates_choose[ipeak,1],coordinates_choose[ipeak,0],str(ipeak), color='b',size= 20)
    pk_all = coordinates - (np.array(fft2abs.shape)/2)
    pk_choose = pk_all[choose,:]
  return pk_choose,pk_all
arr_cln = arr_c_g_mmmm

##handles

In [ ]:
vmin = 0 # @param
vmax = 0.0000001 # @param
threshold = 0.0000002 # @param
choose = [6,9,10] # @param

## Visualization for Check

In [ ]:
fft2abs = fft2show(arr_cln,vmin,vmax)
pk_choose,pk_all = fft2pkfnd(fft2abs,threshold,choose)
print(pk_choose)
print(pk_all)

#Remove extrinsic drift effect

In [ ]:
def generate_sine_wave_from_k(k_value,  arr_shape,phi=0):
  """
  Generates a 2D sine wave based on a k-value (spatial frequency vector).

  Args:
    k_value: A 2-element numpy array representing the centered k-vector
             (ky_c, kx_c) in pixel units from the FFT.
    arr_shape: A tuple (image_height, image_width) representing the shape
               of the desired output image in pixels.
    phi: Optional phase offset in radians. Defaults to 0.

  Returns:
    A 2D numpy array representing the sine wave.
  """
  # Get the centered k-vector components
  ky_c = k_value[0]
  kx_c = k_value[1]
  image_height, image_width = arr_shape
  # Create a grid of spatial coordinates (pixel indices)
  y_indices, x_indices = np.indices((image_height, image_width))

  # Calculate spatial frequencies in cycles per pixel
  spatial_freq_x_cpp = kx_c / image_width
  spatial_freq_y_cpp = ky_c / image_height

  # Construct the full argument for the sine wave, including the 2*pi factor and phase
  argument = 2 * np.pi * (spatial_freq_x_cpp * x_indices + spatial_freq_y_cpp * y_indices) + phi

  # Generate the sine wave
  sine_wave = np.sin(argument)

  return sine_wave
def kdispacementmap(arr,k,sig):
  sw = generate_sine_wave_from_k(k,  arr.shape,0)
  cw = generate_sine_wave_from_k(k,  arr.shape,np.pi/2)
  sprod = gaussian_filter(sw*arr,sig)
  cprod = gaussian_filter(cw*arr,sig)
  xprod = cprod +sprod*1j
  return xprod
def visxprod(xprod):
  fig,axs = plt.subplots(2,2,figsize=(10, 10))

  #sw = generate_sine_wave_from_k(k,  aff_br.shape,0)
  tns = ['Re','Img','theta','radial']
  isbs = [0,0,1,1]
  jsbs = [0,1,0,1]
  funcs = [np.real, np.imag, np.angle, np.abs]
  cmaps = ['PuOr','vanimo','twilight','plasma']
  for isb,jsb,func,tn,cmap in zip(isbs,jsbs,funcs,tns,cmaps):
    reimg = func(xprod)
    #if tn == 'theta':
     # reimg = unwrap_phase(reimg)
    #else:
    vmin,vmax = auto_clim(reimg, method='percentile')
    im = axs[isb,jsb].imshow(reimg, cmap=cmap,vmin=vmin,vmax=vmax)
    plt.colorbar(im, ax=axs[isb,jsb])
    axs[isb,jsb].set_title(tn)
    axs[isb,jsb].set_xticks([])
    axs[isb,jsb].set_yticks([])
  return fig, axs

def auto_clim(data, method='percentile', lower=1, upper=99):
  """
  data: 2D array
  method: 'percentile' or 'std'
  """
  if method == 'percentile':
      vmin, vmax = np.percentile(data, [lower, upper])
  elif method == 'std':
      mean = np.mean(data)
      std = np.std(data)
      vmin, vmax = mean - std, mean + std
  else:
      raise ValueError("method must be 'percentile' or 'std'")
  return vmin, vmax

In [ ]:
ik = 1 # @param
k = pk_choose[ik]/3
sig = 10 # @param
xprod = kdispacementmap(arr_cln,k,sig)
visxprod(xprod)
plt.suptitle("k"+str(ik))

In [ ]:
ks_Latt = pk_choose
ks_CDW = pk_choose/3

In [ ]:
def wrap_phase(unwrapped_phase):
  """
  Wraps unwrapped phase data into the range of (-pi, pi].

  Args:
    unwrapped_phase: A numpy array containing the unwrapped phase data.

  Returns:
    A numpy array containing the wrapped phase data.
  """
  wrapped_phase = np.mod(unwrapped_phase + np.pi, 2 * np.pi) - np.pi
  return wrapped_phase
from skimage.restoration import unwrap_phase

In [ ]:

sig = 20 # @param
angle_restores = []

for ik in range(3):
  k_Latt = ks_Latt[ik]
  k_CDW = ks_CDW[ik]
  angle_restore = (unwrap_phase(np.angle(kdispacementmap(arr_cln,k_CDW,sig)))
   - unwrap_phase(np.angle(kdispacementmap(arr_cln,k_Latt,sig)))/3)
  angle_restores.append(angle_restore)




In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15, 15))
tns = ['k1','k2','k3']
yns = ['CDW','atom','CDW_compensate']
for ik,tn in zip(range(3),tns):
    reimg = angle_restores[ik]
    reimg = wrap_phase(reimg)
    axs[ik].imshow(reimg, cmap='twilight')
    axs[ik].set_title(tn)
    axs[ik].set_xticks([])
    axs[ik].set_yticks([])

In [ ]:
class CropWindow:
    def __init__(self, *, xywh=None, xlimylim=None, rlist=None):
        if xywh is not None:
            self.x, self.y, self.w, self.h = xywh
        elif xlimylim is not None:
            (x_min, x_max), (y_min, y_max) = xlimylim
            self.x, self.y = x_min, y_min
            self.w, self.h = x_max - x_min, y_max - y_min
        elif rlist is not None:
            xs = [p[0] for p in rlist]
            ys = [p[1] for p in rlist]
            self.x, self.y = min(xs), min(ys)
            self.w, self.h = max(xs) - min(xs), max(ys) - min(ys)
        else:
            raise ValueError("One of xywh, xlimylim, rlist must be provided.")

    # --------------------
    # 변환 메서드들
    # --------------------
    def to_xywh(self):
        return (self.x, self.y, self.w, self.h)

    def to_xlimylim(self):
        return (self.x, self.x + self.w), (self.y, self.y + self.h)

    def to_rlist(self):
        return [
            (self.x, self.y),
            (self.x + self.w, self.y),
            (self.x + self.w, self.y + self.h),
            (self.x, self.y + self.h),
        ]

    # 보기 좋게 출력
    def __repr__(self):
        return f"CropWindow(xywh={self.to_xywh()})"


In [ ]:
cw1 = CropWindow(xywh=[400, 200, 300, 400])
foo = cw1.to_rlist()
print(foo)
print(foo[0])
foo.append(foo[0])
r5 = np.array(foo)
xlimylim = cw1.to_xlimylim()

In [ ]:
reimg

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15, 15))
tns = ['k1','k2','k3']
yns = ['CDW','atom','CDW_compensate']
fitRng = []
for ik,tn in zip(range(3),tns):
    reimg = angle_restores[ik][xlimylim[1][0]:xlimylim[1][1],xlimylim[0][0]:xlimylim[0][1]]
    # reimg = angle_restores[ik]
    reimg = wrap_phase(reimg)

    axs[ik].imshow(reimg, cmap='twilight')
    axs[ik].set_title(tn)
    axs[ik].set_xticks([])
    axs[ik].set_yticks([])
    fitRng.append(reimg)
    # axs[ik].plot(r5[:,0],r5[:,1])
    # axs[ik].set_xlim([xlimylim[0][0],xlimylim[0][1]])
    # axs[ik].set_ylim([xlimylim[1][0],xlimylim[1][1]])

In [ ]:
ik = 0
img = fitRng[ik]
shifting = img.mean(axis=1)
plt.plot(shifting)

not enough good to apply sigmoid fitting.....

In [ ]:
def sigmoid(x, A, L, x0, k):
    """
    Sigmoid function with controllable parameters.

    Parameters:
    - x: Independent variable (numpy array or scalar)
    - A: Lower asymptote (step bottom)
    - L: Upper asymptote (step top)
    - x0: Midpoint (step position)
    - k: Steepness/gradient (step gradient)
    """
    return A + (L - A) / (1 + np.exp(-k * (x - x0)))

In [ ]:
yTipChange = 395
Ls =np.pi*(2/3)*np.array([1,1,-1])
shp = angle_restores[0].shape
x = np.array(range(shp[1]))
A = 0
# L=0
x0 = yTipChange
k = 1000

# plt.plot(x,ysig)

In [ ]:


is_wrap_phase = True # @param {type:"boolean"}
is_crop_axes = False # @param {type:"boolean"}
# is_window_plot = True # @param {type:"boolean"}


fig,axs = plt.subplots(1,3,figsize=(15, 15))
tns = ['k1','k2','k3']
yns = ['CDW','atom','CDW_compensate']
fitRng = []
for ik,tn in zip(range(3),tns):
    # reimg = angle_restores[ik][xlimylim[1][0]:xlimylim[1][1],xlimylim[0][0]:xlimylim[0][1]]
    reimg = angle_restores[ik]
    ysig = sigmoid(x, A, Ls[ik], x0, k)
    reimg = reimg - ysig.reshape(shp[1],1)
    if is_wrap_phase:
      reimg = wrap_phase(reimg)

    axs[ik].imshow(reimg, cmap='twilight')
    axs[ik].set_title(tn)
    axs[ik].set_xticks([])
    axs[ik].set_yticks([])
    fitRng.append(reimg)
    # axs[ik].hlines(yTipChange,0,2047)
    if is_crop_axes:
      axs[ik].set_xlim([xlimylim[0][0],xlimylim[0][1]])
      axs[ik].set_ylim([xlimylim[1][0],xlimylim[1][1]])

In [ ]:
for ik,tn in zip(range(3),tns):
  reimg = angle_restores[ik]
  ysig = sigmoid(x, A, Ls[ik], x0, k)
  angle_restores[ik] = reimg - ysig.reshape(shp[1],1)


#Visualization

Determine guage ground

In [ ]:
def auto_clim(data, method='percentile', lower=1, upper=99):
  """
  data: 2D array
  method: 'percentile' or 'std'
  """
  if method == 'percentile':
      vmin, vmax = np.percentile(data, [lower, upper])
  elif method == 'std':
      mean = np.mean(data)
      std = np.std(data)
      vmin, vmax = mean - std, mean + std
  else:
      raise ValueError("method must be 'percentile' or 'std'")
  return vmin, vmax


In [ ]:
plt.figure(figsize=(10, 10)) # Adjust the size as needed
plt.imshow(arr_cln,cmap='afmhot')
ax = plt.gca()
ax.images[0].set_clim(
    auto_clim(arr_cln, method='percentile', lower=1, upper=99.9)
)
x = 875
y = 730
ax.plot(x,y,"bo")
ax.set_xlim([700,900])
ax.set_ylim([700,900])


In [ ]:
angle_restores_g = []
for ik in range(3):
  angle_restore = angle_restores[ik]
  angle_restore_g = angle_restore - angle_restore[y,x]
  angle_restores_g.append(angle_restore_g)

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15, 15))
tns = ['k1','k2','k3']
yns = ['CDW','atom','CDW_compensate']
for ik,tn in zip(range(3),tns):
    reimg = angle_restores[ik]
    reimg = wrap_phase(reimg)
    axs[ik].imshow(reimg, cmap='twilight')
    axs[ik].set_title(tn)
    axs[ik].set_xticks([])
    axs[ik].set_yticks([])
    axs[ik].plot(x,y,"ko")
    # ax.set_xlim([700,900])
    # ax.set_ylim([700,900])

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(15, 15))
tns = ['k1','k2','k3']
yns = ['CDW','atom','CDW_compensate']
for ik,tn in zip(range(3),tns):
    reimg = angle_restores_g[ik]
    reimg = wrap_phase(reimg)
    axs[ik].imshow(reimg, cmap='twilight')
    axs[ik].set_title(tn)
    axs[ik].set_xticks([])
    axs[ik].set_yticks([])
    axs[ik].plot(x,y,"ko")
    # ax.set_xlim([700,900])
    # ax.set_ylim([700,900])

In [ ]:
reimg = angle_restores_g[0]
# reimg = angle_restores_g[ik]
reimg = wrap_phase(reimg)
fig = plt.figure(figsize=(10, 10))
plt.imshow(reimg, cmap='twilight')
# plt.set_title(tn)
# plt.set_xticks([])
# plt.set_yticks([])
plt.plot(x,y,"ko")

In [ ]:
import numpy as np

## Divide ##
def z3z3split(phi1,phi2):
  # phi1 from [0,2pi)
  # phi2 from [0,2pi)
  phi1 = phi1%(2*np.pi) # R -> [0,2pi)
  phi2 = phi2%(2*np.pi)
  phi1i = phi1/(2*np.pi) # [0,2pi) -> [0,1)
  phi2i = phi2/(2*np.pi)
  z3z31, rphi1 = divmod(phi1i, 1/3)
  z3z32, rphi2 = divmod(phi2i, 1/3)
  z3z3 = [z3z31,z3z32] # {0,1,2} #len2 list
  rphi12 = [rphi1*3,rphi2*3] # [0,1) #len2 list
  return z3z3,rphi12

def rlsplit(rphi12):
  is_close_10 = rphi12[0]>rphi12[1]#r-like|-> T, g-like|-> F
  if not is_close_10:
    rphi12 = [rphi12[1],rphi12[0]]
  rphi12 = np.array(rphi12)
  rphi12 = rphi12.reshape(2,1) #Basis B
  I_C__B = np.array([[1,-1],[0,1]]) #Identity
  rphi12_C = I_C__B@rphi12 #Basis C
  v_c1 = rphi12_C[0,0]
  v_c2 = rphi12_C[1,0]
  baryHHH_foo = np.array([1-v_c1-v_c2,v_c1,v_c2 ])
  baryHHH_foo = baryHHH_foo.reshape(3,1) #order of index: (00),(??),(11)
  return is_close_10,baryHHH_foo

def sixsplit(baryHHH_foo):
  p_BarHHH = np.sort(baryHHH_foo, axis=0)[::-1]
  z6 = np.argsort(baryHHH_foo, axis=0)[::-1]
  p_AffHHH = p_BarHHH[1:]
  I_AffHAC__AffHHH = np.array([[1,2],[1,-1]])
  p_AffHAC = I_AffHAC__AffHHH@p_AffHHH
  p_BarHAC = np.array([1-p_AffHAC[0]-p_AffHAC[1],p_AffHAC[0],p_AffHAC[1]])
  baryHAC_foo = p_BarHAC
  return z6,baryHAC_foo

## Conquer ##
def get_baryHHH(baryHHH_foo,is_close_10):
  baryHHH = np.zeros(4)
  baryHHH[0] = baryHHH_foo[0,0]
  baryHHH[3] = baryHHH_foo[2,0]
  # print('here',not(is_close_10)+1)
  baryHHH[(not(is_close_10))+1] = baryHHH_foo[1,0]
  return baryHHH #order of index: (00),(10),(01),(11)
def get_baryHAC(is_close_10,baryHAC_foo):
  # baryHAC = np.zeros(3)
  baryHAC = baryHAC_foo
  if not(is_close_10):
    baryHAC = baryHAC_foo[[0,2,1],[0]]
  return baryHAC #order of index: H,10L,01L
def phaseInfo(phi1,phi2):
  z3z3,rphi12 = z3z3split(phi1,phi2)
  is_close_10,baryHHH_foo = rlsplit(rphi12)
  z6,baryHAC_foo = sixsplit(baryHHH_foo)
  baryHHH = get_baryHHH(baryHHH_foo,is_close_10)
  baryHAC = get_baryHAC(is_close_10,baryHAC_foo)
  return baryHHH,baryHAC,z3z3,z6

In [ ]:
def phaseInfo2(Phi1,Phi2):
  XY_shape = Phi1.shape
  BaryHHH = np.zeros((XY_shape[0],XY_shape[1],4))
  BaryHAC = np.zeros((XY_shape[0],XY_shape[1],3))
  Z3z3 = np.zeros((XY_shape[0],XY_shape[1],2))
  Z6 = np.zeros((XY_shape[0],XY_shape[1],3))
  for i in range(XY_shape[0]):
    for j in range(XY_shape[1]):
      phi1_now = Phi1[i,j]
      phi2_now = Phi2[i,j]

      baryHHH,baryHAC,z3z3,z6 = phaseInfo(phi1_now,phi2_now)

      BaryHHH[i,j,:] = baryHHH.flatten()
      BaryHAC[i,j,:] = baryHAC.flatten()
      Z3z3[i,j,0] = z3z3[0]
      Z3z3[i,j,1] = z3z3[1]
      Z6[i,j,0] = z6[0]
      Z6[i,j,1] = z6[1]
      Z6[i,j,2] = z6[2]
  return BaryHHH,BaryHAC,Z3z3,Z6

In [ ]:
type(angle_restores_g)
angle_restores_g[ik].shape

In [ ]:
Phi1 = (1/2)*angle_restores_g[0]+(-1/4)*angle_restores_g[1]+(-1/4)*angle_restores_g[2]
Phi2 = (-1/4)*angle_restores_g[0]+(1/2)*angle_restores_g[1]+(-1/4)*angle_restores_g[2]


Phi1 = angle_restores_g[0]
Phi2 = -angle_restores_g[1]

In [ ]:
Info = phaseInfo2(Phi1,Phi2)

In [ ]:
HAC_holder = Info[1]
HAC_argmax = np.argmax(HAC_holder, axis=2)
fig = plt.figure(figsize=(20,20))
plt.imshow(HAC_argmax, origin='lower',cmap='gray')

# plt.plot(250,200,'ro')

In [ ]:
DWThick = .2
dlt = DWThick

In [ ]:
dwBW = Info[1][:,:,0]<DWThick

In [ ]:
def getDWsegment(z6,is_close10):
  class6 = [210,201,120,102,21,12]
  blk = [1012,1102,120,210]
  rlk = [1120,1210,12,102]
  glk = [1021,1201,21,201]
  class12 = np.array([blk,rlk,glk])

  class6this = np.sum(z6*np.array([100,10,1]))
  z6 =  class6 == class6this
  class12this = class6this +1000*is_close10
  classified = class12 == class12this
  true_indices = np.where(classified)
  classified3 = true_indices[0][0]
  return classified3

In [ ]:
Z6 = Info[3]
Is_close10 = Info[0][:,:,2]>0

# ix,iy= 93,22

classified3s = np.zeros(Z6.shape[0:2])
for ix in range(Z6.shape[0]):
  for iy in range(Z6.shape[1]):
    z6 = Z6[ix,iy,:]# input 1
    is_close10 = Is_close10[ix,iy]# input 2
    classified3 = getDWsegment(z6,is_close10)
    classified3s[ix,iy] = classified3

In [ ]:
classified3s_rgb = np.stack((classified3s==1,classified3s==0,classified3s==2),axis=2)

In [ ]:
dw_rgb = classified3s_rgb * dwBW[:, :, np.newaxis]

In [ ]:
Z3z3,Rphi12 =  z3z3split(Phi1,Phi2)
coefs = np.array([[-2/4,4/4],[1/2,1/2],[4/4,-2/4]])
# coefs.shape

chiral = np.einsum('ijk,li->jkl', np.array(Rphi12), coefs)
chiral = chiral[:,:,[2,0,1]]
chiral[:,:,2] = -chiral[:,:,2]

In [ ]:
chiral_ = np.mod(chiral+dlt/2,.5)-dlt/2
chiral_ = -(1/dlt)*(chiral_-dlt/2)
#chiral_[:,:2]=chiral_[:,:2]

In [ ]:
final = chiral_*dw_rgb

In [ ]:
final2 = 1

In [ ]:
for ik in range(3):
  foo = final[:,:,ik]
  print(foo.min(),foo.max())


In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(final.astype(np.float64))
plt.plot(x,y,"wo")

In [ ]:
from skimage.color import rgb2hsv,hsv2rgb
final2 = rgb2hsv(final)
final2[:,:,1] = final2[:,:,2]
final2[:,:,2] = 1
final2 = hsv2rgb(final2)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(final2.astype(np.float64))


In [ ]:
plt.figure(figsize=(10, 10)) # Adjust the size as needed
plt.imshow(arr_cln,cmap='afmhot')
ax = plt.gca()
ax.images[0].set_clim(
    auto_clim(arr_cln, method='percentile', lower=1, upper=99.9)
)
x = 875
y = 730
ax.plot(x,y,"bo")
ax.set_xlim([600,1000])
ax.set_ylim([500,900])

In [ ]:
def get_z3z3hex(Info):
  HHH_argmax = np.argmax(Info[0], axis=2)
  mod01,mod10 = np.divmod(HHH_argmax ,2)
  hex1 = np.mod(Info[2][:,:,0] + mod10,3)
  hex2 = np.mod(Info[2][:,:,1] + mod01,3)
  z3z3hex = np.stack((hex1,hex2),axis=2)
  return z3z3hex

In [ ]:
z3z3hex = get_z3z3hex(Info)

In [ ]:
p_00 = [.5,.5,.5]
p_12 = [.1,.1,.1]
p_21 = [.8,.8,.8]

p_10 = [1,0,0]
p_01 = [0,1,0]
p_22 = [0,0,1]

p_20 = [0,1,1]
p_02 = [1,0,1]
p_11 = [1,1,0]

In [ ]:
colors = {
    "Red Bright": (1.0, 0.302, 0.302),   # #FF4D4D
    "Red Dark":   (0.478, 0.110, 0.110), # #7A1C1C
    "Green Bright": (0.302, 1.0, 0.302), # #4DFF4D
    "Green Dark":   (0.110, 0.478, 0.110), # #1C7A1C
    "Blue Bright":  (0.302, 0.58, 1.0),  # #4D94FF
    "Blue Dark":    (0.110, 0.110, 0.478) # #1C1C7A
}

In [ ]:
p_00 = [.5,.5,.5]
p_12 = [.1,.1,.1]
p_21 = [.8,.8,.8]

p_10 = colors["Red Dark"]
p_01 = colors["Green Dark"]
p_22 = colors["Blue Dark"]

p_20 = colors["Red Bright"]
p_02 = colors["Green Bright"]
p_11 = colors["Blue Bright"]

In [ ]:
clrset = [[p_00,p_01,p_02],[p_10,p_11,p_12],[p_20,p_21,p_22]]
clrset = np.array(clrset)
clrset.shape

In [ ]:
shp = z3z3hex.shape
phase9 = np.zeros((shp[0],shp[1],3))
for ix  in range(shp[0]):
  for iy in range(shp[1]):
    phase9[ix,iy,:] = clrset[int(z3z3hex[ix,iy,0]),int(z3z3hex[ix,iy,1]),:]

In [ ]:
plt.imshow(phase9)

In [ ]:
plt.imshow(z3z3hex[:,:,0])

In [ ]:
plt.imshow(z3z3hex[:,:,1])

In [ ]:
shp

In [ ]:
xlimylim_DWN = ((0,shp[0]),(0,shp[1]))
xlimylim_DV = ((500,750),(1250,1500))
xlimylim_DW = ((525,625),(1400,1500))

xlimylim_D = ((585,625),(1460,1500))
xlimylim_L = ((607,625),(1460,1478))
xlimylims = [xlimylim_DWN,xlimylim_DV,xlimylim_DW,xlimylim_D,xlimylim_L]
xlimylims = xlimylims[::-1]
# wV =CropWindow(xlimylim=xlimylim_DV)
# plt

In [ ]:
# plt.figure(figsize=(10, 10)) # Adjust the size as needed
# plt.imshow(arr_cln,cmap='afmhot')
# ax = plt.gca()
# ax.images[0].set_clim(
#     auto_clim(arr_cln, method='percentile', lower=1, upper=99.9)
# )
# ax.set_xlim(xlimylim_L[0])
# ax.set_ylim(xlimylim_L[1])


In [ ]:
class CropWindow:
    def __init__(self, *, xywh=None, xlimylim=None, rlist=None):
        if xywh is not None:
            self.x, self.y, self.w, self.h = xywh
        elif xlimylim is not None:
            (x_min, x_max), (y_min, y_max) = xlimylim
            self.x, self.y = x_min, y_min
            self.w, self.h = x_max - x_min, y_max - y_min
        elif rlist is not None:
            xs = [p[0] for p in rlist]
            ys = [p[1] for p in rlist]
            self.x, self.y = min(xs), min(ys)
            self.w, self.h = max(xs) - min(xs), max(ys) - min(ys)
        else:
            raise ValueError("One of xywh, xlimylim, rlist must be provided.")

    # --------------------
    # 변환 메서드들
    # --------------------
    def to_xywh(self):
        return (self.x, self.y, self.w, self.h)

    def to_xlimylim(self):
        return (self.x, self.x + self.w), (self.y, self.y + self.h)

    def to_rlist(self):
        return [
            (self.x, self.y),
            (self.x + self.w, self.y),
            (self.x + self.w, self.y + self.h),
            (self.x, self.y + self.h),
        ]
    def to_rlist5(self):
        return np.array([
            (self.x, self.y),
            (self.x + self.w, self.y),
            (self.x + self.w, self.y + self.h),
            (self.x, self.y + self.h),
            (self.x, self.y),
        ])
    # 보기 좋게 출력
    def __repr__(self):
        return f"CropWindow(xywh={self.to_xywh()})"

In [ ]:
is_subplotTranspose = False # @param {type:"boolean"}
nCr =5
nmap =4
msb = nmap
nsb =nCr


if is_subplotTranspose:
  msb,nsb=nsb,msb
  fig,axs = plt.subplots(msb,nsb,figsize=(nsb*5, msb*4))
  axs = axs.transpose()
else:
  fig,axs = plt.subplots(msb,nsb,figsize=(nsb*4, msb*4))

for iCr in range(nCr):
  for imap in range(nmap):
    ax = axs[imap,iCr]
    if imap==0:
      ax.imshow(arr_cln,cmap='afmhot')
      ax.images[0].set_clim(
          auto_clim(arr_cln, method='percentile', lower=1, upper=99.9)
      )
    elif imap==1:
      ax.imshow(phase9)
    elif imap==2:
      ax.imshow(HAC_argmax,cmap='gray')
    elif imap==3:
      ax.imshow(final2.astype(np.float64))

    #

    #


    ax.set_xlim(xlimylims[iCr][0])
    ax.set_ylim(xlimylims[iCr][1])
    # if iCr>0:
    #   # xlimylims[iCr-1]
    #   rlist5 = CropWindow(xlimylim=xlimylims[iCr-1]).to_rlist5()
    #   print(rlist5[:,0])
    #   print(rlist5[:,1])
    #   ax.plot(rlist5[:,0],rlist5[:,1],'w')
    for iCr2 in range(nCr):
      rlist5 = CropWindow(xlimylim=xlimylims[iCr2]).to_rlist5()
      ax.plot(rlist5[:,0],rlist5[:,1])
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(final2.astype(np.float64))
ax = plt.gca()
ax.set_xlim(xlimylims[iCr][0])
ax.set_ylim(xlimylims[iCr][1])

In [ ]:
xlimylim_DVR = ((560,690),(1310,1440))
# xlimylim_DVL = ((1100,1350),(100,350))
xlimylim_DVL = ((1860,1990),(260,390))
xlimylim_DVs =[xlimylim_DVR,xlimylim_DVL]
xlimylims = xlimylim_DVs

In [ ]:
is_subplotTranspose = True # @param {type:"boolean"}
nCr =2
nmap =4
msb = nmap
nsb =nCr


if is_subplotTranspose:
  msb,nsb=nsb,msb
  fig,axs = plt.subplots(msb,nsb,figsize=(nsb*5, msb*4))
  axs = axs.transpose()
else:
  fig,axs = plt.subplots(msb,nsb,figsize=(nsb*4, msb*4))

for iCr in range(nCr):
  for imap in range(nmap):
    ax = axs[imap,iCr]
    if imap==0:
      ax.imshow(arr_cln,cmap='afmhot')
      ax.images[0].set_clim(
          auto_clim(arr_cln, method='percentile', lower=1, upper=99.9)
      )
    elif imap==1:
      ax.imshow(phase9)
    elif imap==2:
      ax.imshow(HAC_argmax,cmap='gray')
    elif imap==3:
      ax.imshow(final2.astype(np.float64))

    #

    #


    ax.set_xlim(xlimylims[iCr][0])
    ax.set_ylim(xlimylims[iCr][1])
    # if iCr>0:
    #   # xlimylims[iCr-1]
    #   rlist5 = CropWindow(xlimylim=xlimylims[iCr-1]).to_rlist5()
    #   print(rlist5[:,0])
    #   print(rlist5[:,1])
    #   ax.plot(rlist5[:,0],rlist5[:,1],'w')
    for iCr2 in range(nCr):
      rlist5 = CropWindow(xlimylim=xlimylims[iCr2]).to_rlist5()
      ax.plot(rlist5[:,0],rlist5[:,1])
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
shifty=1000
xlimylim_DWN = ((0,shp[0]),(0,shp[1])) #full
xlimylim_DV = ((500,500+512),(shifty,512+shifty)) #40nm
xlimylim_DW =  ((500,500+256),(256+shifty,256+256+shifty)) #20nm

xlimylim_D = ((500+64,500+128+64),(256+shifty,256+128+shifty)) #10nm
xlimylim_L = ((500+64,500+64+64),(256+shifty,256+64+shifty))#5nm
xlimylims = [xlimylim_DWN,xlimylim_DV,xlimylim_DW,xlimylim_D,xlimylim_L]
xlimylims = xlimylims[::-1]

In [ ]:
stn = '110K(sig=20)'
tns = ['5nm','10nm','20nm','40nm','full(=80nm)']

In [ ]:
is_subplotTranspose = False # @param {type:"boolean"}
nCr =5
nmap =4
msb = nmap
nsb =nCr


if is_subplotTranspose:
  msb,nsb=nsb,msb
  fig,axs = plt.subplots(msb,nsb,figsize=(nsb*5, msb*4))
  axs = axs.transpose()
else:
  fig,axs = plt.subplots(msb,nsb,figsize=(nsb*4, msb*4))

for iCr in range(nCr):
  for imap in range(nmap):
    ax = axs[imap,iCr]
    if imap==0:
      ax.imshow(arr_cln,cmap='afmhot')
      ax.images[0].set_clim(
          auto_clim(arr_cln, method='percentile', lower=1, upper=99.9)
      )
      ax.set_title(tns[iCr])
    elif imap==1:
      ax.imshow(phase9)
    elif imap==2:
      ax.imshow(HAC_argmax,cmap='gray')
    elif imap==3:
      ax.imshow(final2.astype(np.float64))

    #

    #


    ax.set_xlim(xlimylims[iCr][0])
    ax.set_ylim(xlimylims[iCr][1])
    # if iCr>0:
    #   # xlimylims[iCr-1]
    #   rlist5 = CropWindow(xlimylim=xlimylims[iCr-1]).to_rlist5()
    #   print(rlist5[:,0])
    #   print(rlist5[:,1])
    #   ax.plot(rlist5[:,0],rlist5[:,1],'w')
    for iCr2 in range(nCr):
      rlist5 = CropWindow(xlimylim=xlimylims[iCr2]).to_rlist5()
      ax.plot(rlist5[:,0],rlist5[:,1])
    ax.set_xticks([])
    ax.set_yticks([])
plt.suptitle(stn)